In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp

In [26]:
data = pd.read_csv("fitting.txt", delimiter = " ",skiprows = 1, header = None, names = ['x','y','sigma_y'])
data = data.sample(frac=1)
data_training = data.head(10)
data_test = data.tail(10)

x_training = data_training["x"] 
y_training = data_training["y"]
sigma_training = data_training["sigma_y"]

x_test = data_test["x"]
y_test = data_test["y"]
sigma_test = data_test["sigma_y"]

In [47]:
def model(x,params):
    n_dim = len(params)
    y = 0
    for i in range(n_dim):
        y += params[i] * (x**i)
    return y

def loglk(x,y,params):
    modelo = model(x,params)
    d = y - modelo
    d = d/sigma_y
    d = -0.5 * np.sum(d**2) + np.sum(np.log(1/(2*np.pi*sigma_y*sigma_y)**0.5))
    return d

def logprior(params):
    n_dim = len(params)
    p = -np.inf
    if (params<=1).all() and (params>=-1).all():
        p = n_dim*np.log(2)
    return p

In [48]:
def metropolis(x,y,n_dim,N=50000):
    lista_params = [np.random.uniform(-1,1,n_dim)]
    sigma_params = np.ones(len(lista_params))*0.01
    logposterior = [0]
    for i in range(1,N):
        propuesta  = lista_params[i-1] + np.random.normal(size=n_dim)*sigma_params
    
        logposterior_viejo = loglk(x, y, lista_params[i-1]) + logprior(lista_params[i-1])
        logposterior_nuevo = loglk(x, y, propuesta) + logprior(propuesta)

        r = min(1,np.exp(logposterior_nuevo-logposterior_viejo))
        alpha = np.random.random()
        if(alpha<r):
            lista_params.append(propuesta)
            logposterior.append(logposterior_nuevo)
        else:
            lista_params.append(lista_params[i-1])
            logposterior.append(logposterior_viejo)
    
    lista_params = np.array(lista_params)
    lista_params = lista_params[N//10:,:] 
    logposterior = np.array(logposterior)
    logposterior = logposterior[N//10:]
    return lista_params, logposterior

In [49]:
def MSE(y,y_model,sigma_y):
    y_total = y-y_model
    y_total = (y_total**2)/(sigma_y**2)
    total = np.sum(y_total)/10
    return total

In [50]:
mse_training = np.ones(10)
for n_dim in range(1,11):
    lista_params, logposterior = metropolis(x,y,n_dim)
    y_model = model(x_training,lista_params)
    mse_n = MSE(y_training,y_model,sigma_training)
    mse_training[n_dim-1] = mse_n

ValueError: operands could not be broadcast together with shapes (2,) (10,) 